#**Finetuning the latest GEMMA-2 models**

Installing the packages


In [ ]:
!pip install -q -U bitsandbytes==0.43.3
!pip install -q -U peft==0.12.0
!pip install -q -U trl==0.10.1
!pip install -q -U accelerate==0.34.2
!pip install -q -U datasets==3.0.0
!pip install -q -U transformers==4.44.2


Importing the necessary libraries


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

Setting the Hugging Face APII token

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

Model selection and configuration

In [ ]:
model_id = "google/datagemma-rag-27b-it"
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

Initializations

In [ ]:
tokenizer = AutoTokenizer.from_pretrained (model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                              quantization_config=bnb_config,
                                              device_map={"":0},
                                              token=os.environ ['HF_TOKEN'])

Text generation and detokenization

In [ ]:
text = "Quote: Good things take time"
device = "cuda:0"
inputs = tokenizer (text, return_tensors="pt").to(device)
outputs = model.generate (**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Enabling Weights & Biases

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

LoRA configuration


In [ ]:
þora_config = LoraConfig(
r == 8,
target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
"gate_proj", "up_proj", "down_proj"],
task_type = "CAUSAL_LM",
)

Loading dataset from Hugging Face

In [ ]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer (samples["quote"]), batched=True)

Training

In [ ]:
data['train']['quote']

In [ ]:
def formatting_func(example):
text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
return [text]

In [ ]:
lora_config = LoraConfig(
  r = 8,
  target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
  "gate_proj", "up_proj", "down_proj"],
  task_type = "CAUSAL_LM",
  )

Training Initialization

In [ ]:
trainer = SFTTrainer(
model=model,
train_dataset=data["train"],
args=transformers. TrainingArguments(
per_device_train_batch_size=1,
gradient_accumulation_steps=4,
warmup_steps=2,
max_steps=100,
learning_rate=2e-4,
fp16=True,
logging_steps=1,
output_dir="outputs",
optim="paged_adamw_8bit"
),
peft_config=lora_config,
formatting_func=formatting_func,
)

In [ ]:
trainer.train()

Testing by data generation

In [ ]:
text = "Quote: Be yourself;"
device = "cuda:0"
inputs = tokenizer (text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print (tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
text = "Quote: The opposite of love is not hate,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
text = "Quote: So many books; so little time"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate (**inputs, max_new_tokens=20)
print (tokenizer.decode(outputs[0], skip_special_tokens=True))

Evaluating BLEU and ROUGE scores

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load your fine-tuned model and tokenizer
model_id = "your_fine_tuned_model_id"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Prepare evaluation data
eval_texts = ["text 1", "text 2", "text 3"]
eval_references = [["reference 1.1", "reference 1.2"], ["reference 2.1"], ["reference 3.1"]]

# Generate text
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
generated_texts = generator(eval_texts, max_length=100, num_beams=4)

# Calculate metrics
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# BLEU
bleu_scores = []
for generated, references in zip(generated_texts, eval_references):
    bleu_scores.append(sentence_bleu(references, generated[0], smoothing_function=SmoothingFunction().method1))
average_bleu = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU:", average_bleu)

# ROUGE
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(generated_texts, eval_references)
print("ROUGE scores:", scores)

Successfully finetuned and implemented.